In [3]:
import os
import sys
from ftplib import FTP, error_perm
import patoolib
import glob
import patoolib
import pandas as pd
currentPath=os.getcwd()

#os.chdir('/Users/dare.olufunmilayo/GIT/Scripts/Python/downloads/extracts/csvPath')
#print(currentPath)

/Users/dare.olufunmilayo/GIT/Scripts/Python/downloads/extracts/csvPath


In [3]:
def weatherDataDownloader(stationId,startYear,endYear,url="ftp.pyclass.com",user="student@pyclass.com",passwd="student123"):
    ftp = FTP(url)
    ftp.login(user,passwd)    
    if os.path.basename(currentPath) == "downloads":
        downloadDir=currentPath
    else:
        downloadDir=currentPath+"/downloads/"
    if not os.path.exists(downloadDir):
        os.makedirs(downloadDir)
    try:
        os.chdir(downloadDir)
        listOfFiles = glob.glob('*.gz')
    except FileNotFoundError:
        print('FileNotFound error')
        pass  
    
    for Year in range(startYear , endYear + 1):
            #Full path of the FTP direcorty
        fullPath = '/Data/%s/%s-%s.gz' % (Year,stationId, Year)
        filename = os.path.basename(fullPath) 
        if listOfFiles != []:
            print('Files already exist in this directory')
            break
        try:
            with open(filename, "wb") as file:
                ftp.retrbinary('RETR %s' % fullPath , file.write)                     
            print('%s downloaded successfully' % filename)
        except error_perm:
            print('%s is not available for download' % filename)
            os.remove(filename)
        except TypeError:
            print('If you quoted the years, remove the quotes and try again')
    ftp.close()
        
# What happens if i try to download a wrong file and files already exist in the download directory?
# I need to create another directory inside the downloads folder to reflect the specific station and year range that is being downloaded
# Serialize the user name and password using pickle


In [28]:

def weatherDataExctractor():
    
    if os.path.basename(currentPath) == "downloads":
        downloadedDir=currentPath
    else:
        downloadedDir=currentPath+"/downloads/"
    extractedDir=downloadedDir + "/extracts"
    if not os.path.exists(extractedDir):
        os.makedirs(extractedDir)
    try:
        os.chdir(downloadedDir)
        listOfFiles = glob.glob('*.gz')
    except FileNotFoundError:
        print('FileNotFound error')
    if os.listdir(extractedDir) == []:        
        for file in listOfFiles:
                patoolib.extract_archive(file, outdir=extractedDir)
                print('files extratced check %s' % extractedDir)
    else:
        print('Extracted files already exist. review the files contained in %s' % extractedDir)
        sys.exit()
        #Add additional Field to file
    print('Adding stationID column to all the files...')
    os.chdir(extractedDir)
    listExtractedFiles= glob.glob('*')
    print(os.getcwd())
    csvPath = extractedDir+"/csvPath"
    if not os.path.exists(csvPath):
        os.makedirs(csvPath)
    if os.listdir(csvPath) != []:
        print('Csv files already exist')
        sys.exit()
    for extractedFile in listExtractedFiles:
        try:
            df = pd.read_csv(extractedFile, sep='\s+', header=None)
            df['stationID']= [extractedFile.rsplit('-', 1)[0]]*df.shape[0] 
            df.to_csv(csvPath+"/"+extractedFile+".csv", index=0, header=None)
        except Exception:
            print('Some empty data exist, but some of the files have been extracted. Check the directory' )
        except OSError:
            print('you are not in the correct directory')
        except EmptyDataError:
            print('empty data error occured')

    
def concatenateFiles(inDir):
    #os.chdir(inDir)
    concatenatedDir=inDir+"/concatenated"
    if not os.path.exists(concatenatedDir):
        os.makedirs(concatenatedDir)
    listofFiles= os.listdir(inDir)
    DFfiles= []
    DFcolumns = ['Year', 'Month', 'Day', 'Hour', 'Temp', 'DewTemp', 'Pressure', 'WinDir', 'WinSpeed', 'Sky', 'Precipitation1', 'Precipitation6', 'StationID']
    #os.chdir(concatenatedDir)
    for file in listofFiles:
        df= pd.read_csv(file, header=None)
        DFfiles.append(df)
    concatDF=pd.concat(DFfiles, axis=0)
    concatDF.columns(DFcolumns)
    concatDF.to_csv(concatenatedDir, index = 0)

    

In [7]:
weatherDataDownloader('029070-99999',1901,1926)

Files already exist in this directory


In [ ]:
weatherDataExctractor()


In [ ]:
concatenateFiles('/Users/dare.olufunmilayo/GIT/Scripts/Python/downloads/extracts/csvPath')

In [27]:
print(os.getcwd())
print(os.listdir())

/Users/dare.olufunmilayo/GIT/Scripts/Python/downloads/extracts/csvPath
['029070-99999-1901.csv', '029070-99999-1902.csv', '029070-99999-1903.csv', '029070-99999-1904.csv', '029070-99999-1905.csv', 'cancatenated']


In [15]:
os.chdir('/Users/dare.olufunmilayo/GIT/Scripts/Python/downloads/extracts/csvPath')

In [38]:
df= pd.read_csv('/Users/dare.olufunmilayo/GIT/Scripts/Python/downloads/extracts/csvPath/029070-99999-1901.csv',header = None)
df.head

<bound method NDFrame.head of         0   1   2   3    4     5      6    7    8   9     10    11  \
0     1901   1   1   6  -78 -9999  10200  270  159   8 -9999 -9999   
1     1901   1   1  13  -72 -9999  10200  290   82   4 -9999 -9999   
2     1901   1   1  20  -94 -9999  10200    0    0   8 -9999 -9999   
3     1901   1   2   6  -61 -9999  10183  180   82   8 -9999 -9999   
4     1901   1   2  13  -56 -9999  10176  180   98   8 -9999 -9999   
5     1901   1   2  20  -28 -9999  10175  180   98   8 -9999 -9999   
6     1901   1   3   6  -67 -9999  10170  200   98   6 -9999 -9999   
7     1901   1   3  13  -33 -9999  10174  230  118   8 -9999 -9999   
8     1901   1   3  20  -28 -9999  10174  230  118   8 -9999 -9999   
9     1901   1   4   6  -33 -9999  10231    0    0   8 -9999 -9999   
10    1901   1   4  13  -44 -9999  10226  230   82   8 -9999 -9999   
11    1901   1   4  20  -39 -9999  10223  200  118   8 -9999 -9999   
12    1901   1   5   6    0 -9999  10182  270   41   4 -9999